In [1]:
import os
import tqdm as notebook_tqdm
from dotenv import load_dotenv

import dsp
import dspy
from dotdict import dotdict
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFinetune, BootstrapFewShotWithRandomSearch

load_dotenv()
unify_api_key = os.getenv("UNIFY_KEY")



In [2]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=1000, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

In [3]:
print(devset)
print(trainset)

[Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 'gold_titles': {'Cangzhou', 'Qionghai'}}) (input_keys={'question'}), Example({'question': 'Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?', 'answer': 'National Hockey League', 'gold_titles': {'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}}) (input_keys={'question'}), Example({'question': 'The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay Lightning of the National Hockey League (NHL)?', 'answer': 'Steve Yzerman', 'gold_titles': {'2006–07 Detroit Red Wings season', 'Steve Yzerman'}}) (input_keys={'question'}), Example({'question': 'What river is near the Crichton Collegiate Church?', 'answer': 'the River Tyne', 'gold_titles': {'Crichton Collegiate Church', 'Crichton Castle'}}) (input_keys={'questi

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class QnA(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return dspy.Prediction(answer=prediction.answer)

In [5]:
gemma = dsp.Unify(
    endpoint="gemma-7b-it@anyscale",
    max_tokens=150,
    api_key=unify_api_key,
)

claude = dsp.Unify(
    endpoint="claude-3-haiku@anthropic",
    max_tokens=150,
    api_key=unify_api_key,
)

unify_router = dsp.Unify(
    endpoint="router@q:1|c:1.92e+00|t:7.45e-05|i:1.00e-06",
    max_tokens=150,
    api_key=unify_api_key,
)

turbo = dsp.Unify(
    endpoint="gpt-3.5-turbo@openai", 
    max_tokens=150,
    api_key=unify_api_key,

)

dspy.settings.configure(lm=claude)

In [6]:
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

num_threads = 24

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)



In [8]:
compiled_qna = teleprompter.compile(QnA(), trainset=trainset)

 40%|████      | 8/20 [00:10<00:16,  1.37s/it]


Bootstrapped 4 full traces after 9 examples in round 0.


In [9]:
evaluate_hotpot = Evaluate(devset=devset[:1000], metric=validate_context_and_answer, num_threads=num_threads, display_progress=True, display_table=5)
evaluate_hotpot(compiled_qna, metric=validate_context_and_answer)

Average Metric: 18 / 50  (36.0): 100%|██████████| 50/50 [00:13<00:00,  3.78it/s]


,question,example_answer,gold_titles,pred_answer,validate_context_and_answer
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",George McPhee,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}",River Nith,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",Alfred the Great,False


36.0

In [10]:
turbo.inspect_history(n=3)




Answer questions with short factoid answers.

---

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: In what year was the club founded that played Manchester City in the 1972 FA Charity Shield
Answer: 1874

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas fou

'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nQuestion: At My Window was released by which American singer-songwriter?\nAnswer: John Townes Van Zandt\n\nQuestion: "Everything Has Changed" is a song from an album released under which record label ?\nAnswer: Big Machine Records\n\nQuestion: In what year was the club founded that played Manchester City in the 1972 FA Charity Shield\nAnswer: 1874\n\nQuestion: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?\nAnswer: Aleem Sarwar Dar\n\nQuestion: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?\nAnswer: "Outfield of Dreams"\n\nQuestion: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?\nAnswer: Aleksandr Danilovich Aleksandrov\n\nQuestion: The Organisation that allows a community to influence their operation or use and to enjoy t

In [ ]:
tp = BootstrapFinetune(metric=validate_context_and_answer, teacher_settings=dict({'lm': turbo}))

In [ ]:
finetuned_qna = tp.compile(student=QnA(), trainset=trainset, target='google/flan-t5-base')